In [ ]:
import ohawf
import pandas as pd
from httpx import get
from headlines import *
from pathlib import Path
from random import randint
from collections import namedtuple
from apiclient.discovery import build
from sqlitedict import SqliteDict as sqldict
from datetime import date, datetime, timedelta
from shutil import copyfile, move, copytree, rmtree

In [ ]:
h1("Constrain time-ranges")

Week = namedtuple("Week", "number, start, end")

# Set Week One
start_date = date(1970, 8, 27)  # Set to birthday or UTS-Zero date(1970, 1, 1).

# Constrain random week selection to between:
no_older_than = date(2010, 11, 7)
no_newer_than = date(2013, 12, 31)

print(f"Using start date: {start_date:%A}, {start_date}")

# Display what day of week you chose, and week-zero start date.
weekday = start_date.weekday()
days_to_sunday = 6 - weekday
start_week = start_date - timedelta(days=days_to_sunday)
print(f"Week-zero starts: {start_week:%A}, {start_week}")

# Show how many weeks there have been since start date.
current_week = start_week
week_number = 0
while current_week < datetime.today().date():
    current_week = current_week + timedelta(days=7)
    week_number += 1
weeks_to_date = week_number
print(f"It has been {weeks_to_date} weeks since start.")

h2("Table of tuples (this is your life)")
current_week = start_week
week_number = 0
table = []
while current_week < datetime.today().date():
    end_of_week = current_week + timedelta(days=6)
    row = Week(f"{week_number}", f"{current_week}", f"{end_of_week}")
    table.append(row)
    current_week = current_week + timedelta(days=7)
    week_number += 1
print("Start")
for item in table[:3]:
    print(item)
print("...")
for item in table[-3:]:
    print(item)
print("End")

h2("Selecting random week")
# Caluclate the eariest week number for the randint range.
for i, week in enumerate(table):
    start = datetime.strptime(week.start, "%Y-%m-%d").date()
    if start >= no_older_than:
        break
earliest_number = int(week.number)
for i, week in enumerate(table):
    end = datetime.strptime(week.start, "%Y-%m-%d").date()
    if end >= no_newer_than:
        break
latest_number = int(week.number)

while True:
    random_week = table[randint(earliest_number, latest_number)]
    if random_week not in [int(x.name) for x in Path("photos/source").glob("*")]:
        break

print(f"Random week between {start} and {end} is:")
print(f"{random_week}.")

namespace = "photos"
database = f"{namespace}/db"
source = f"{namespace}/source"

Path(namespace).mkdir(exist_ok=True, parents=True)
Path(database).mkdir(exist_ok=True, parents=True)
Path(source).mkdir(exist_ok=True, parents=True)

h3("Done")

In [ ]:
h1("Logging in")
scopes = [
    "https://www.googleapis.com/auth/photoslibrary.readonly",
    "https://www.googleapis.com/auth/photoslibrary.appendonly",
]
creds = ohawf.get(scopes=scopes, cli=True)
service = build("photoslibrary", "v1", credentials=creds, static_discovery=False)
h3("Done")

In [ ]:
h1("Save Album Meta (not belonging)")

result = service.albums().list(pageSize=50).execute()
npt = result.get("nextPageToken")
albumdb = f"{database}/albums_raw.db"

with sqldict(albumdb) as db:
    db[npt] = result
    db.commit()

while npt:
    result = service.albums().list(pageSize=50, pageToken=npt).execute()
    npt = result.get("nextPageToken")
    with sqldict(albumdb) as db:
        db[npt] = result
        db.commit()

table = []
with sqldict(albumdb) as db:
    for npt in db:
        if npt:
            data = db[npt]
            for album in data["albums"]:
                table.append((album["title"], album["id"]))

df = pd.DataFrame(table, columns=["Name", "ID"])
df.to_csv("photos/db/albums.csv", index=False)

h3("Done")

In [ ]:
paramdict = {"all": (True, False), "favs": (True, True), "main": (False, False)}
weektupes = []
start = random_week.start
end = random_week.end
Week = namedtuple("Week", "number, start, end, arch, fav")
for name in paramdict:
    atuple = paramdict[name]
    arch, fav = atuple
    aweek = Week(random_week.number, start, end, arch, fav)
    weektupes.append((name, aweek))

for weektupe in weektupes:
    name, aweek = weektupe
    dbname = f"{aweek.number}-{name}.db"
    print(dbname)

def search(npt, start, end, archive=True, favorites=False):
    sY, sM, sD = start.split("-")
    eY, eM, eD = end.split("-")

    args = {
        "pageSize": 100,
        "pageToken": npt,
        "filters": {
            "includeArchivedMedia": archive,
            "dateFilter": {
                "ranges": [
                    {
                        "startDate": {"year": sY, "month": sM, "day": sD},
                        "endDate": {"year": eY, "month": eM, "day": eD},
                    }
                ]
            },
        },
    }
    if favorites:
        args["filters"]["featureFilter"] = {"includedFeatures": ["FAVORITES"]}
    return args

In [ ]:
h1("Save all, favs & main lists for week")

for weektupe in weektupes:
    name, aweek = weektupe
    search_args = weektupe[1][1:]
    dbname = f"{database}/{aweek.number}-{name}.db"
    h2(dbname)

    # if not Path(dbname).exists():
    args = search(None, *search_args)
    print(args)
    results = service.mediaItems().search(body=args).execute()
    with sqldict(dbname) as db:
        db[str(args)] = results
        db.commit()
        h3("Saved Inital Results")
        if "nextPageToken" in results:
            npt = results["nextPageToken"]
            c = 1
            while npt:
                args = search(npt, *search_args)
                results = service.mediaItems().search(body=args).execute()
                db[str(args)] = results
                db.commit()
                print(c, end=" ")
                c += 1
                if "nextPageToken" in results:
                    npt = results["nextPageToken"]
                else:
                    npt = None
                    h3("Paged results saved")
                    break

h3("Done")

In [ ]:
for weektupe in weektupes:
    name, aweek = weektupe
    search_args = weektupe[1][1:]
    dbname = f"{database}/{aweek.number}-{name}.db"
    if name == "all":
        break
print(dbname)

num_folder = f"{source}/{aweek.number}"

h2(f"Downloading full source images from {random_week.number}")

Path(f"{num_folder}/archive").mkdir(exist_ok=True, parents=True)

with sqldict(dbname) as db:
    for i, npt in enumerate(db):
        results = db[npt]
        print(i + 1, end=" ")
        for item in results["mediaItems"]:
            if item["mimeType"].split("/")[0] == "image":
                oldname = item["filename"]
                filename = f"{item['id']}____{oldname}"
                full_path = f"{num_folder}/{filename}"
                if not Path(full_path).exists():
                    use_url = f"{item['baseUrl']}=d"
                    response = get(use_url, timeout=30)
                    with open(full_path, "wb") as fh:
                        fh.write(response.content)
h3("Done")